# The preprocessing script needs to be modified to incorporate the l1000 idx. This has been done in the copy of the repo called "moler_reference_copy"

In [ ]:
# molecule_generation preprocess l1000_biaae/INPUT_DIR l1000_biaae/OUTPUT_DIR l1000_biaae/TRACE_DIR --motif_vocabulary_provided='/data/ongh0068/guacamol/trace_dir/metadata.pkl.gz' --using_lincs

In [1]:
%load_ext autoreload
%autoreload 2
from dataset import LincsDataset
from model import BaseModel
from dataset import MolerDataset, MolerData
from utils import pprint_pyg_obj
from torch_geometric.loader import DataLoader
import torch
from model_utils import get_params

dataset = LincsDataset(
    root = '/data/ongh0068', 
    raw_moler_trace_dataset_parent_folder = '/data/ongh0068/guacamol/trace_dir',
    output_pyg_trace_dataset_parent_folder = '/data/ongh0068/l1000/l1000_biaae/already_batched',
    split = 'valid_0',
    gene_exp_controls_file_path = '/data/ongh0068/l1000/l1000_biaae/lincs/robust_normalized_controls.npz',
    gene_exp_tumour_file_path = '/data/ongh0068/l1000/l1000_biaae/lincs/robust_normalized_tumors.npz',
    lincs_csv_file_path = '/data/ongh0068/l1000/l1000_biaae/lincs/experiments_filtered.csv',
)

2023-03-01 09:25:44.947161: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading controls gene expression...
Loading tumour gene expression...
Loading csv...


In [2]:
loader = DataLoader(dataset, batch_size=1, shuffle=False, follow_batch = [
    'correct_edge_choices',
    'correct_edge_types',
    'valid_edge_choices',
    'valid_attachment_point_choices',
    'correct_attachment_point_choice',
    'correct_node_type_choices',
    'original_graph_x',
    'correct_first_node_type_choices'
])


for batch in loader:
#     batch.cuda()
    break

In [3]:
# import pandas as pd
# df = pd.read_csv('/data/ongh0068/l1000/l1000_biaae/lincs/experiments_filtered.csv')

In [4]:
# batch.gene_expressions.float()

In [5]:
params = get_params(dataset)
model = BaseModel(params = params, dataset = dataset, using_lincs = True)


In [7]:
model._run_step(batch)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1000x1490 and 1810x512)

In [9]:
from aae import AAE
params = get_params(dataset)
params['gene_exp_condition_mlp']['input_feature_dim']= 832 + 978
model = AAE(params = params, dataset = dataset, using_lincs = True)

In [12]:
model._run_step(batch)

torch.Size([1000, 832])


MoLeROutput(first_node_type_logits=tensor([[-0.1043, -0.0248,  0.0225,  ..., -0.0320, -0.1041,  0.0722],
        [-0.0914, -0.0383,  0.0395,  ..., -0.0266, -0.1080,  0.1036],
        [-0.1012, -0.0363,  0.0247,  ..., -0.0794, -0.1089,  0.0963],
        ...,
        [-0.1146, -0.0241,  0.0391,  ..., -0.0517, -0.0990,  0.0737],
        [-0.0770, -0.0391,  0.0624,  ..., -0.0521, -0.0657,  0.0821],
        [-0.0761,  0.0009,  0.0105,  ..., -0.0269, -0.1058,  0.0774]],
       grad_fn=<AddmmBackward0>), node_type_logits=tensor([[ 0.0118, -0.0310,  0.0146,  ...,  0.0508,  0.0738, -0.0200],
        [-0.0024, -0.0362,  0.0479,  ...,  0.0448,  0.0456, -0.0328],
        [-0.0570, -0.0457,  0.0376,  ...,  0.0315,  0.0517, -0.0284],
        ...,
        [-0.0160, -0.0441,  0.0297,  ...,  0.0438,  0.0697, -0.0286],
        [-0.0267, -0.0372,  0.0390,  ...,  0.0399,  0.0642, -0.0492],
        [-0.0290, -0.0444,  0.0435,  ...,  0.0237,  0.0700, -0.0388]],
       grad_fn=<AddmmBackward0>), edge_candida

# Train/Validation/Test split

The original biaae paper splits it into 50/25/25 randomly https://github.com/insilicomedicine/BiAAE/blob/71a2df2c0d7ed5d775a560e66da029ce63b46007/dataloader/lincs_dl.py#L145. We do the same here.

In [31]:
import pandas as pd
df = pd.read_csv('/data/ongh0068/l1000/l1000_biaae/lincs/experiments_filtered.csv', index_col = False)

In [32]:
import numpy as np
# df['ControlIndices'].values.astype(float)

def str_to_int(row):
    row['ControlIndices'] = np.asarray(row['ControlIndices'].split(' '), dtype = np.int32)
    row['TumourIndices'] = np.asarray(row['TumourIndices'].split(' '), dtype = np.int32)

    return row 
df = df.apply(lambda x: str_to_int(x), axis = 1)

In [36]:
df['ControlIndices'].values.astype(np.int32)

ValueError: setting an array element with a sequence.

In [51]:
import random
batch_idx = df['ControlIndices'].values[np.array([2,3,5,7])]
control_idx = [random.randint(0,len(arr)) for arr in batch_idx]
torch.from_numpy(robust_norm_controls['genes'])[control_idx].shape

torch.Size([4, 978])

In [25]:
len(df[df['CellLine'] == 'A375'].SMILES), len(df[df['CellLine'] == 'A375'].SMILES.unique())

(7996, 3644)

In [29]:
df['cl_t_d'] = df['CellLine'] + "_" + df['Time'].astype(str) + '_' + df['Dose'].astype(str)

In [49]:
from collections import Counter

Counter(df['cl_t_d'].tolist())

Counter({'A375_6.0_10.0': 2592,
         'A549_6.0_10.0': 2593,
         'A549_24.0_10.0': 2239,
         'ASC_24.0_10.0': 1191,
         'HA1E_6.0_10.0': 2302,
         'HA1E_24.0_10.0': 1125,
         'HCC515_6.0_10.0': 2494,
         'HCC515_24.0_10.0': 762,
         'HEPG2_6.0_10.0': 1930,
         'HT29_6.0_10.0': 2586,
         'MCF7_6.0_10.0': 2869,
         'MCF7_24.0_10.0': 2875,
         'NPC_24.0_10.0': 1351,
         'PC3_6.0_10.0': 2777,
         'PC3_24.0_10.0': 2833,
         'PHH_24.0_10.0': 864,
         'SKB_24.0_10.0': 1187,
         'VCAP_6.0_10.0': 2611,
         'VCAP_24.0_10.0': 2597,
         'MCF7_6.0_0.3704': 104,
         'MCF7_6.0_1.1111': 76,
         'MCF7_6.0_3.3333': 81,
         'MCF7_24.0_0.3704': 104,
         'MCF7_24.0_1.1111': 75,
         'MCF7_24.0_3.3333': 80,
         'NKDBA_24.0_0.3704': 30,
         'NKDBA_24.0_3.3333': 28,
         'NKDBA_24.0_10.0': 29,
         'A375_24.0_10.0': 792,
         'A375_6.0_0.15625': 11,
         'A375_6.0_0.62

# To check on: whether there is a point in stratification

In [52]:
# df = df.rename(columns = {'Unnamed: 0': 'original_idx'})

In [25]:
from sklearn.model_selection import train_test_split


df_train, df_test = train_test_split(df, test_size=0.5, random_state=42)
df_valid, df_test = train_test_split(df_test, test_size=0.5, random_state=42)


In [26]:
df_train.to_csv('/data/ongh0068/l1000/l1000_biaae/INPUT_DIR/train.csv', index = False)
df_valid.to_csv('/data/ongh0068/l1000/l1000_biaae/INPUT_DIR/valid.csv', index = False)
df_test.to_csv('/data/ongh0068/l1000/l1000_biaae/INPUT_DIR/test.csv', index = False)

In [1]:
# pip install molecule-generation


In [12]:
from moler_reference import molecule_generation

In [13]:
import gzip
import pickle

with gzip.open('/data/ongh0068/l1000/l1000_playground/TRACE_DIR/train_0/train_0.pkl.gz', 'rb') as f:
     train= pickle.load(f)

In [19]:
train[0].mol

In [15]:
for trace_step in train[0]:
    break

# investigate npz

1. npz holds the actual 978 dim vector corresponding to the gene expression
2. df holds the indexes that allow us to retrieve the 978 dim vectors from the npz ['genes'] + dose
3. note that each row in the df will have multiple possible indexes for the control and for the tumour gene expressions because multiple experiments were run to repeat them for averaging. This is worked around by randomly selecting one
4. the difference between the gene expressions are calculated
5. the difference is concatenated with the dose (retrieved from df)

given a trace step -> use idx to retrieve 


In [45]:
from numpy import load

In [11]:
pip install -U scikit-learn==0.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 16.0 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1517 lines of output]
      Partial import of sklearn during the build process.
      /tmp/pip-install-mjez1dzm/scikit-learn_f77f2a60dede4388ad1f420e9acad728/setup.py:242: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils_status_migration.html
      
      
        from numpy.distutils.core import setup
      INFO: blas_opt_info:
      INFO: blas_armpl_info:
      INFO:

Failed to build scikit-learn
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
  Running setup.py install for scikit-learn ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for scikit-learn did not run successfully.
  │ exit code: 1
  ╰─> [1517 lines of output]
      Partial import of sklearn during the build process.
      /tmp/pip-install-mjez1dzm/scikit-learn_f77f2a60dede4388ad1f420e9acad728/setup.py:242: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils_status_migration

Note: you may need to restart the kernel to use updated packages.


In [46]:
robust_norm_controls_fp ='/data/ongh0068/l1000/l1000_biaae/lincs/robust_normalized_controls.npz'
robust_norm_controls = load(robust_norm_controls_fp, allow_pickle = True)
robust_norm_tumours_fp = '/data/ongh0068/l1000/l1000_biaae/lincs/robust_normalized_tumors.npz'
robust_norm_tumours = load(robust_norm_tumours_fp, allow_pickle = True)

In [10]:
df.ControlIndices[0]

'12373 12374 12375 12376 12377 12378 12379 12380 12381 12382 12383 12384 12385 12386 12387 12388 12389 12390 12391 12392 12393 12394 12395 12396 12397 12398 12399 12400 12401 12402 12403'

In [17]:
df.iloc[0]

Unnamed: 0                                                                                         0
CellLine                                                                                        A375
Time                                                                                             6.0
Dose                                                                                            10.0
ControlIndices                                     12373 12374 12375 12376 12377 12378 12379 1238...
TumourIndices                                                                   343818 343819 343820
SMILES                                                     CC1CS(=O)(=O)CCN1N=Cc1ccc([N+](=O)[O-])o1
DrugIndex_maccs                                                                                 6172
DrugIndex_pharmacophore2d720                                                                    1394
DrugIndex_morgan                                                                           

In [14]:
robust_norm_controls['genes'].shape

(27264, 978)

In [18]:
robust_norm_controls['gene_ids'][12373 ]

'CPC015_A375_6H_X1_B4_DUO52HI53LO:A17'

In [23]:
robust_norm_tumours['gene_ids'][343818 ]

'CPC015_A375_6H_X1_B4_DUO52HI53LO:L10'

In [25]:

diff = robust_norm_tumours['genes'][343818] - robust_norm_controls['genes'][12373 ]



In [26]:
diff

array([ 1.63608225e-02, -3.81278007e-01, -3.31983878e+00, -2.07374867e-01,
       -1.34353939e-01, -2.49763373e-01,  1.41466254e-01, -2.17668632e-01,
       -1.35045061e-01,  4.48599037e-01, -5.80755937e-01,  2.51387666e-01,
        4.60170875e-02, -1.06545257e-01, -3.08708085e-02,  1.12309285e-01,
       -4.18417672e-02, -1.65675381e-01, -3.39899007e-02, -2.29070503e-01,
        1.40583532e+00, -3.32450781e-01, -1.93277816e-01, -6.54316371e-01,
        3.83517900e-01,  3.47108708e-01, -1.39630231e-01, -3.17162896e-01,
        3.64239795e-01,  4.54819837e-02, -2.11859376e-01, -2.67956548e-02,
       -4.14444200e-01, -1.90313251e-01, -1.86518440e+00, -1.62204792e-02,
        2.40208477e-01,  3.36859396e-01, -1.84915675e-02, -2.45687388e-01,
       -4.37774861e-01, -2.02805471e-01, -2.80213520e-01,  4.99731335e-02,
        1.95472267e-01, -3.93587721e-01,  2.44379683e-01,  3.44857803e-01,
       -1.88993297e-01, -1.98536175e-01, -2.78868439e-01, -2.27672102e-01,
        1.24092105e-01,  

In [ ]:



import gzip
import pickle
with gzip.open('/data/ongh0068/guacamol/trace_dir/metadata.pkl.gz', 'rb') as f:
     metadata= pickle.load(f)

In [12]:
import gzip
import pickle
with gzip.open('/data/ongh0068/guacamol/trace_dir/metadata.pkl.gz', 'rb') as f:
     metadata= pickle.load(f)

In [14]:
metadata

{'train_atom_num_distribution': Counter({6: 336,
          34: 40874,
          27: 66292,
          25: 72550,
          35: 34964,
          33: 46184,
          31: 56533,
          29: 62913,
          39: 17456,
          19: 37592,
          45: 6471,
          22: 60754,
          38: 20886,
          40: 15003,
          24: 71287,
          23: 65695,
          26: 71646,
          32: 51955,
          52: 2642,
          28: 65911,
          30: 61117,
          18: 30548,
          20: 45918,
          17: 23917,
          36: 29518,
          48: 4304,
          37: 24309,
          21: 52936,
          42: 10994,
          16: 18312,
          43: 9000,
          47: 4743,
          49: 3509,
          41: 12442,
          50: 3395,
          14: 9962,
          15: 13716,
          54: 1966,
          44: 8048,
          59: 365,
          46: 5762,
          10: 2169,
          12: 5178,
          13: 6994,
          56: 1115,
          9: 1410,
          11: 3488,
     

In [10]:
to_inspect = '/data/ongh0068/l1000/l1000_playground/TRACE_DIR/metadata.pkl.gz'
import gzip
import pickle
with gzip.open(to_inspect, 'rb') as f:
     metadata= pickle.load(f)

In [11]:
metadata

{'train_atom_num_distribution': Counter({19: 19, 26: 1}),
 'train_atom_type_distribution': Counter({'C': 207,
          'S': 19,
          'O': 80,
          'N': 43,
          'N+': 19,
          'O-': 19}),
 'sa_score_mean': 3.629005285795853,
 'sa_score_stddev': 0.0767669901210126,
 'clogp_mean': 0.6096299999999999,
 'clogp_stddev': 0.1349951002814547,
 'mol_weight_mean': 290.763462151,
 'mol_weight_stddev': 16.153515595799902,
 'qed_mean': 0.45733159875566737,
 'qed_stddev': 0.00539037385474397,
 'bertz_mean': 617.4968317339059,
 'bertz_stddev': 48.21199785464212,
 'feature_extractors': [<molecule_generation.chem.atom_feature_utils.AtomTypeFeatureExtractor at 0x7f1efdd252b0>,
 'motif_vocabulary': MotifVocabulary(vocabulary={'C1=CC=CC=C1': 0, 'C1=CC=NC=C1': 1, 'NC=O': 2, 'C1CCNCC1': 3, 'C1CNCCN1': 4, 'FC(F)F': 5, 'C1CCNC1': 6, 'O=CO': 7, 'C1=CNN=C1': 8, 'O=[N+][O-]': 9, 'CCC': 10, 'C1=CN=CN=C1': 11, 'C1CCCCC1': 12, 'C1COCCN1': 13, 'C1=CSC=C1': 14, 'C1=CC=C2NC=CC2=C1': 15, 'CCO': 16,